<a href="https://colab.research.google.com/github/senadityadev343/celebal-technology-assignments/blob/main/Celebal_Technologies_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Install Required Libraries
!pip install pandas sqlite3 pyarrow openpyxl plotly

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [2]:
# Cell 2: Import Libraries
import pandas as pd
import sqlite3
import json
import os
from datetime import datetime, timedelta
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from google.colab import files
import shutil

In [3]:
# Cell 3: Create Directory Structure
def create_directory_structure():
    """Create required directories"""
    directories = [
        '/content/data',
        '/content/output/Customer',
        '/content/output/Product',
        '/content/output/CustomerWithAddress',
        '/content/logs'
    ]
    for directory in directories:
        os.makedirs(directory, exist_ok=True)
        print(f"✅ Created directory: {directory}")
create_directory_structure()


✅ Created directory: /content/data
✅ Created directory: /content/output/Customer
✅ Created directory: /content/output/Product
✅ Created directory: /content/output/CustomerWithAddress
✅ Created directory: /content/logs


In [4]:
# Cell 4: Create Sample Customer Data
def create_customer_data():
    """Create realistic customer data"""
    np.random.seed(42)
    # Generate 2000 customers
    customers = pd.DataFrame({
        'CustomerId': range(1, 2001),
        'CustomerName': [f'Customer_{i:04d}' for i in range(1, 2001)],
        'Email': [f'customer{i}@email.com' for i in range(1, 2001)],
        'Phone': [f'+1-{np.random.randint(100,999)}-{np.random.randint(100,999)}-{np.random.randint(1000,9999)}'
                 for _ in range(2000)],
        'RegistrationDate': pd.date_range(start='2020-01-01', periods=2000, freq='D')[:2000],
        'Status': np.random.choice(['Active', 'Inactive', 'Pending'], 2000, p=[0.7, 0.2, 0.1]),
        'CreditScore': np.random.randint(300, 850, 2000)
    })
    return customers
customers_df = create_customer_data()
print(f"✅ Created {len(customers_df)} customer records")
customers_df.head()


✅ Created 2000 customer records


,CustomerId,CustomerName,Email,Phone,RegistrationDate,Status,CreditScore
0,1,Customer_0001,customer1@email.com,+1-202-535-1860,2020-01-01,Active,438
1,2,Customer_0002,customer2@email.com,+1-370-206-6191,2020-01-02,Active,528
2,3,Customer_0003,customer3@email.com,+1-800-120-6734,2020-01-03,Active,573
3,4,Customer_0004,customer4@email.com,+1-221-566-5426,2020-01-04,Inactive,738
4,5,Customer_0005,customer5@email.com,+1-558-187-9322,2020-01-05,Active,417


In [5]:
# Cell 5: Create Sample Product Data
def create_product_data():
    """Create realistic product data"""
    np.random.seed(42)
    categories = ['Electronics', 'Clothing', 'Books', 'Home', 'Sports', 'Beauty']
    products = pd.DataFrame({
        'ProductId': range(1, 201),
        'ProductName': [f'Product_{i:03d}' for i in range(1, 201)],
        'Category': np.random.choice(categories, 200),
        'Price': np.round(np.random.uniform(10, 1000, 200), 2),
        'StockQuantity': np.random.randint(0, 100, 200),
        'Supplier': [f'Supplier_{np.random.randint(1, 20)}' for _ in range(200)],
        'LaunchDate': pd.date_range(start='2019-01-01', periods=200, freq='W')[:200]
    })
    return products
products_df = create_product_data()
print(f"✅ Created {len(products_df)} product records")
products_df.head()


✅ Created 200 product records


,ProductId,ProductName,Category,Price,StockQuantity,Supplier,LaunchDate
0,1,Product_001,Home,226.58,93,Supplier_17,2019-01-06
1,2,Product_002,Sports,562.52,61,Supplier_1,2019-01-13
2,3,Product_003,Books,409.80,68,Supplier_6,2019-01-20
3,4,Product_004,Sports,74.24,75,Supplier_6,2019-01-27
4,5,Product_005,Sports,261.38,15,Supplier_12,2019-02-03


In [6]:
# Cell 6: Create Customer Address Data
def create_address_data():
    """Create customer address data"""
    np.random.seed(42)

    cities = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix',
              'Philadelphia', 'San Antonio', 'San Diego', 'Dallas', 'San Jose']
    states = ['NY', 'CA', 'IL', 'TX', 'AZ', 'PA', 'TX', 'CA', 'TX', 'CA']

    addresses = pd.DataFrame({
        'CustomerId': range(1, 2001),
        'Address': [f'{np.random.randint(1, 9999)} {np.random.choice(["Main St", "Oak Ave", "Park Rd", "First St", "Second Ave"])}'
                   for _ in range(2000)],
        'City': np.random.choice(cities, 2000),
        'State': np.random.choice(states, 2000),
        'ZipCode': [f'{np.random.randint(10000, 99999)}' for _ in range(2000)],
        'Country': ['USA'] * 2000
    })

    return addresses

addresses_df = create_address_data()
print(f"✅ Created {len(addresses_df)} address records")
addresses_df.head()


✅ Created 2000 address records


,CustomerId,Address,City,State,ZipCode,Country
0,1,7271 First St,San Antonio,CA,25690,USA
1,2,861 Park Rd,San Antonio,CA,11909,USA
2,3,5192 Second Ave,Philadelphia,CA,90032,USA
3,4,5735 Oak Ave,Chicago,TX,22062,USA
4,5,467 Park Rd,Philadelphia,CA,85386,USA


In [7]:
# Cell 7: Create SQLite Database
def create_sqlite_database():
    """Create SQLite database with sample data"""
    conn = sqlite3.connect('/content/data/database.sqlite')

    # Save tables to SQLite
    customers_df.to_sql('Customer', conn, if_exists='replace', index=False)
    products_df.to_sql('Product', conn, if_exists='replace', index=False)

    # Verify tables
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    print("✅ Database created successfully!")
    print(f"Tables created: {[table[0] for table in tables]}")

    # Show record counts
    customer_count = pd.read_sql("SELECT COUNT(*) as count FROM Customer", conn).iloc[0]['count']
    product_count = pd.read_sql("SELECT COUNT(*) as count FROM Product", conn).iloc[0]['count']

    print(f"Customer records: {customer_count}")
    print(f"Product records: {product_count}")

    conn.close()

create_sqlite_database()


✅ Database created successfully!
Tables created: ['Customer', 'Product']
Customer records: 2000
Product records: 200


In [8]:
# Cell 8: Create Configuration Files
def create_config_files():
    """Create threshold and other configuration files"""

    # Create threshold.json
    threshold_config = {
        'threshold': 1000,
        'description': 'Minimum record count threshold for data copy',
        'created_date': datetime.now().isoformat(),
        'version': '1.0'
    }

    with open('/content/data/threshold.json', 'w') as f:
        json.dump(threshold_config, f, indent=2)

    # Create customer_address.csv
    addresses_df.to_csv('/content/data/customer_address.csv', index=False)

    # Create pipeline configuration
    pipeline_config = {
        'pipelines': {
            'threshold_pipeline': {
                'enabled': True,
                'schedule': 'daily',
                'output_format': 'json'
            },
            'foreach_pipeline': {
                'enabled': True,
                'tables': ['Customer', 'Product'],
                'output_format': 'json'
            },
            'join_pipeline': {
                'enabled': True,
                'output_format': 'parquet'
            }
        }
    }

    with open('/content/data/pipeline_config.json', 'w') as f:
        json.dump(pipeline_config, f, indent=2)

    print("✅ Configuration files created:")
    print("- threshold.json")
    print("- customer_address.csv")
    print("- pipeline_config.json")

create_config_files()


✅ Configuration files created:
- threshold.json
- customer_address.csv
- pipeline_config.json


In [9]:
# Cell 9: Data Validation and Summary
def validate_data():
    """Validate created data"""
    print("=== DATA VALIDATION SUMMARY ===")

    # Check files exist
    files_to_check = [
        '/content/data/database.sqlite',
        '/content/data/threshold.json',
        '/content/data/customer_address.csv',
        '/content/data/pipeline_config.json'
    ]

    for file_path in files_to_check:
        if os.path.exists(file_path):
            size = os.path.getsize(file_path)
            print(f"✅ {file_path} ({size} bytes)")
        else:
            print(f"❌ {file_path} - NOT FOUND")

    # Database validation
    conn = sqlite3.connect('/content/data/database.sqlite')

    print("\n=== DATABASE VALIDATION ===")
    print("Customer table sample:")
    print(pd.read_sql("SELECT * FROM Customer LIMIT 3", conn))

    print("\nProduct table sample:")
    print(pd.read_sql("SELECT * FROM Product LIMIT 3", conn))

    conn.close()

    print("\n=== CSV VALIDATION ===")
    csv_sample = pd.read_csv('/content/data/customer_address.csv')
    print(f"Address CSV shape: {csv_sample.shape}")
    print(csv_sample.head(3))

validate_data()


=== DATA VALIDATION SUMMARY ===
✅ /content/data/database.sqlite (217088 bytes)
✅ /content/data/threshold.json (156 bytes)
✅ /content/data/customer_address.csv (81782 bytes)
✅ /content/data/pipeline_config.json (375 bytes)

=== DATABASE VALIDATION ===
Customer table sample:
   CustomerId   CustomerName                Email            Phone  \
0           1  Customer_0001  customer1@email.com  +1-202-535-1860   
1           2  Customer_0002  customer2@email.com  +1-370-206-6191   
2           3  Customer_0003  customer3@email.com  +1-800-120-6734   

      RegistrationDate  Status  CreditScore  
0  2020-01-01 00:00:00  Active          438  
1  2020-01-02 00:00:00  Active          528  
2  2020-01-03 00:00:00  Active          573  

Product table sample:
   ProductId  ProductName Category   Price  StockQuantity     Supplier  \
0          1  Product_001     Home  226.58             93  Supplier_17   
1          2  Product_002   Sports  562.52             61   Supplier_1   
2          3  Pr

In [11]:
# Cell 10: Create Data Visualization
def create_data_overview():
    """Create visualizations of the sample data"""

    # Customer status distribution
    fig1 = px.pie(customers_df, names='Status', title='Customer Status Distribution')
    fig1.show()

    # Product price distribution
    fig2 = px.histogram(products_df, x='Price', nbins=20, title='Product Price Distribution')
    fig2.show()

    # Customer registration over time
    customers_df['Month'] = customers_df['RegistrationDate'].dt.to_period('M')
    monthly_reg = customers_df.groupby('Month').size().reset_index(name='Count')
    monthly_reg['Month'] = monthly_reg['Month'].astype(str)

    fig3 = px.line(monthly_reg, x='Month', y='Count', title='Customer Registrations Over Time')
    fig3.update_layout(xaxis_tickangle=45)
    fig3.show()

    print("✅ Data overview visualizations created!")

create_data_overview()

✅ Data overview visualizations created!


In [12]:
# Cell 1: Setup and Imports
import pandas as pd
import sqlite3
import json
import os
from datetime import datetime
import shutil

# Verify data exists
if not os.path.exists('/content/data/database.sqlite'):
    print("❌ Database not found! Please run 01_Data_Setup.ipynb first")
else:
    print("✅ Database found")


✅ Database found


In [13]:
# Cell 2: Threshold Pipeline Class
class ThresholdPipeline:
    def __init__(self, db_path='/content/data/database.sqlite'):
        self.db_path = db_path
        self.log_file = '/content/logs/threshold_pipeline.log'
        self.execution_id = datetime.now().strftime('%Y%m%d_%H%M%S')

    def log(self, message):
        """Log pipeline execution"""
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        log_entry = f"[{timestamp}] [{self.execution_id}] {message}"
        print(log_entry)

        with open(self.log_file, 'a') as f:
            f.write(log_entry + '\n')

    def read_threshold(self):
        """Read threshold value from JSON file"""
        try:
            with open('/content/data/threshold.json', 'r') as f:
                config = json.load(f)
                threshold = config['threshold']
                self.log(f"✅ Threshold loaded: {threshold}")
                return threshold
        except Exception as e:
            self.log(f"❌ Error reading threshold: {str(e)}")
            raise e

    def count_customer_records(self):
        """Count records in Customer table"""
        try:
            conn = sqlite3.connect(self.db_path)
            query = "SELECT COUNT(*) as record_count FROM Customer"
            result = pd.read_sql(query, conn)
            record_count = result.iloc[0]['record_count']
            conn.close()

            self.log(f"✅ Customer record count: {record_count}")
            return record_count
        except Exception as e:
            self.log(f"❌ Error counting records: {str(e)}")
            raise e

    def create_date_folder_structure(self):
        """Create date-based folder structure"""
        now = datetime.now()
        date_path = now.strftime('%Y/%m/%d')
        full_path = f'/content/output/Customer/{date_path}'

        os.makedirs(full_path, exist_ok=True)
        self.log(f"✅ Created folder structure: {full_path}")
        return full_path

    def copy_customer_data(self, output_path):
        """Copy customer data to JSON file"""
        try:
            conn = sqlite3.connect(self.db_path)
            query = "SELECT * FROM Customer"
            customer_data = pd.read_sql(query, conn)
            conn.close()

            # Create filename with timestamp
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f'customer_data_{timestamp}.json'
            full_file_path = os.path.join(output_path, filename)

            # Save as JSON
            customer_data.to_json(full_file_path, orient='records', indent=2)

            self.log(f"✅ Customer data copied to: {full_file_path}")
            self.log(f"✅ Records copied: {len(customer_data)}")

            return full_file_path, len(customer_data)

        except Exception as e:
            self.log(f"❌ Error copying data: {str(e)}")
            raise e

    def execute_pipeline(self):
        """Execute the complete threshold pipeline"""
        self.log("🚀 Starting Threshold Pipeline")

        try:
            # Step 1: Read threshold
            threshold = self.read_threshold()

            # Step 2: Count records
            record_count = self.count_customer_records()

            # Step 3: Check threshold condition
            if record_count > threshold:
                self.log(f"✅ Threshold condition met: {record_count} > {threshold}")

                # Step 4: Create folder structure
                output_path = self.create_date_folder_structure()

                # Step 5: Copy data
                file_path, copied_records = self.copy_customer_data(output_path)

                result = {
                    'status': 'SUCCESS',
                    'threshold': threshold,
                    'record_count': record_count,
                    'copied_records': copied_records,
                    'output_file': file_path,
                    'execution_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                }

                self.log("🎉 Pipeline completed successfully")

            else:
                self.log(f"❌ Threshold condition not met: {record_count} <= {threshold}")
                result = {
                    'status': 'SKIPPED',
                    'threshold': threshold,
                    'record_count': record_count,
                    'reason': 'Record count below threshold',
                    'execution_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                }

            return result

        except Exception as e:
            self.log(f"💥 Pipeline failed: {str(e)}")
            return {
                'status': 'FAILED',
                'error': str(e),
                'execution_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }


In [14]:
# Cell 3: Execute Pipeline
print("=== TASK 1: THRESHOLD-BASED PIPELINE ===")

pipeline = ThresholdPipeline()
result = pipeline.execute_pipeline()


=== TASK 1: THRESHOLD-BASED PIPELINE ===
[2025-07-21 10:02:55] [20250721_100255] 🚀 Starting Threshold Pipeline
[2025-07-21 10:02:55] [20250721_100255] ✅ Threshold loaded: 1000
[2025-07-21 10:02:55] [20250721_100255] ✅ Customer record count: 2000
[2025-07-21 10:02:55] [20250721_100255] ✅ Threshold condition met: 2000 > 1000
[2025-07-21 10:02:55] [20250721_100255] ✅ Created folder structure: /content/output/Customer/2025/07/21
[2025-07-21 10:02:55] [20250721_100255] ✅ Customer data copied to: /content/output/Customer/2025/07/21/customer_data_20250721_100255.json
[2025-07-21 10:02:55] [20250721_100255] ✅ Records copied: 2000
[2025-07-21 10:02:55] [20250721_100255] 🎉 Pipeline completed successfully


In [16]:
# Cell 4: Display Results
print("\n=== PIPELINE EXECUTION RESULTS ===")
# Convert NumPy int64 to standard Python int before dumping to JSON
result['record_count'] = int(result['record_count'])
print(json.dumps(result, indent=2))


=== PIPELINE EXECUTION RESULTS ===
{
  "status": "SUCCESS",
  "threshold": 1000,
  "record_count": 2000,
  "copied_records": 2000,
  "output_file": "/content/output/Customer/2025/07/21/customer_data_20250721_100255.json",
  "execution_time": "2025-07-21 10:02:55"
}


In [17]:
# Cell 5: Validate Output
if result['status'] == 'SUCCESS':
    print("\n=== OUTPUT VALIDATION ===")

    # Check if file exists
    output_file = result['output_file']
    if os.path.exists(output_file):
        print(f"✅ Output file exists: {output_file}")

        # Check file size
        file_size = os.path.getsize(output_file)
        print(f"✅ File size: {file_size} bytes")

        # Load and display sample data
        with open(output_file, 'r') as f:
            sample_data = json.load(f)
            print(f"✅ Records in file: {len(sample_data)}")
            print("\nSample records:")
            for i, record in enumerate(sample_data[:3]):
                print(f"Record {i+1}: {record}")
    else:
        print(f"❌ Output file not found: {output_file}")



=== OUTPUT VALIDATION ===
✅ Output file exists: /content/output/Customer/2025/07/21/customer_data_20250721_100255.json
✅ File size: 454775 bytes
✅ Records in file: 2000

Sample records:
Record 1: {'CustomerId': 1, 'CustomerName': 'Customer_0001', 'Email': 'customer1@email.com', 'Phone': '+1-202-535-1860', 'RegistrationDate': '2020-01-01 00:00:00', 'Status': 'Active', 'CreditScore': 438}
Record 2: {'CustomerId': 2, 'CustomerName': 'Customer_0002', 'Email': 'customer2@email.com', 'Phone': '+1-370-206-6191', 'RegistrationDate': '2020-01-02 00:00:00', 'Status': 'Active', 'CreditScore': 528}
Record 3: {'CustomerId': 3, 'CustomerName': 'Customer_0003', 'Email': 'customer3@email.com', 'Phone': '+1-800-120-6734', 'RegistrationDate': '2020-01-03 00:00:00', 'Status': 'Active', 'CreditScore': 573}


In [18]:
# Cell 6: Test with Different Thresholds
print("\n=== TESTING WITH DIFFERENT THRESHOLDS ===")

# Test with lower threshold
test_threshold = 500
with open('/content/data/threshold.json', 'w') as f:
    json.dump({'threshold': test_threshold}, f, indent=2)

print(f"Testing with threshold: {test_threshold}")
test_pipeline = ThresholdPipeline()
test_result = test_pipeline.execute_pipeline()
print(f"Result: {test_result['status']}")

# Test with higher threshold
test_threshold = 5000
with open('/content/data/threshold.json', 'w') as f:
    json.dump({'threshold': test_threshold}, f, indent=2)

print(f"Testing with threshold: {test_threshold}")
test_pipeline2 = ThresholdPipeline()
test_result2 = test_pipeline2.execute_pipeline()
print(f"Result: {test_result2['status']}")

# Reset to original threshold
with open('/content/data/threshold.json', 'w') as f:
    json.dump({'threshold': 1000}, f, indent=2)



=== TESTING WITH DIFFERENT THRESHOLDS ===
Testing with threshold: 500
[2025-07-21 10:04:39] [20250721_100439] 🚀 Starting Threshold Pipeline
[2025-07-21 10:04:39] [20250721_100439] ✅ Threshold loaded: 500
[2025-07-21 10:04:39] [20250721_100439] ✅ Customer record count: 2000
[2025-07-21 10:04:39] [20250721_100439] ✅ Threshold condition met: 2000 > 500
[2025-07-21 10:04:39] [20250721_100439] ✅ Created folder structure: /content/output/Customer/2025/07/21
[2025-07-21 10:04:39] [20250721_100439] ✅ Customer data copied to: /content/output/Customer/2025/07/21/customer_data_20250721_100439.json
[2025-07-21 10:04:39] [20250721_100439] ✅ Records copied: 2000
[2025-07-21 10:04:39] [20250721_100439] 🎉 Pipeline completed successfully
Result: SUCCESS
Testing with threshold: 5000
[2025-07-21 10:04:39] [20250721_100439] 🚀 Starting Threshold Pipeline
[2025-07-21 10:04:39] [20250721_100439] ✅ Threshold loaded: 5000
[2025-07-21 10:04:39] [20250721_100439] ✅ Customer record count: 2000
[2025-07-21 10:04:

In [19]:
# Cell 7: Folder Structure Visualization
def show_folder_structure():
    """Display the created folder structure"""
    print("\n=== FOLDER STRUCTURE ===")

    def print_tree(directory, prefix="", max_depth=3, current_depth=0):
        if current_depth > max_depth:
            return

        items = []
        try:
            items = sorted(os.listdir(directory))
        except PermissionError:
            print(f"{prefix}❌ Permission denied")
            return
        except FileNotFoundError:
            print(f"{prefix}❌ Directory not found")
            return

        for i, item in enumerate(items):
            if item.startswith('.'):
                continue

            item_path = os.path.join(directory, item)
            is_last = i == len(items) - 1
            current_prefix = "└── " if is_last else "├── "
            print(f"{prefix}{current_prefix}{item}")

            if os.path.isdir(item_path):
                next_prefix = prefix + ("    " if is_last else "│   ")
                print_tree(item_path, next_prefix, max_depth, current_depth + 1)

    print_tree("/content/output")

show_folder_structure()



=== FOLDER STRUCTURE ===
├── Customer
│   └── 2025
│       └── 07
│           └── 21
├── CustomerWithAddress
└── Product


In [20]:
# Cell 8: Pipeline Monitoring Dashboard
import matplotlib.pyplot as plt
import pandas as pd

def create_monitoring_dashboard():
    """Create a simple monitoring dashboard"""
    print("\n=== PIPELINE MONITORING DASHBOARD ===")

    # Read log file
    if os.path.exists('/content/logs/threshold_pipeline.log'):
        with open('/content/logs/threshold_pipeline.log', 'r') as f:
            logs = f.readlines()

        print(f"📊 Total log entries: {len(logs)}")
        print("Recent log entries:")
        for log in logs[-5:]:
            print(log.strip())

    # Create execution summary
    execution_summary = pd.DataFrame([
        {'Metric': 'Pipeline Executions', 'Value': 3},
        {'Metric': 'Successful Executions', 'Value': 2},
        {'Metric': 'Failed Executions', 'Value': 0},
        {'Metric': 'Skipped Executions', 'Value': 1}
    ])

    print("\n📈 Execution Summary:")
    print(execution_summary.to_string(index=False))

create_monitoring_dashboard()

print("\n✅ Task 1 completed successfully!")



=== PIPELINE MONITORING DASHBOARD ===
📊 Total log entries: 20
Recent log entries:
[2025-07-21 10:04:39] [20250721_100439] 🎉 Pipeline completed successfully
[2025-07-21 10:04:39] [20250721_100439] 🚀 Starting Threshold Pipeline
[2025-07-21 10:04:39] [20250721_100439] ✅ Threshold loaded: 5000
[2025-07-21 10:04:39] [20250721_100439] ✅ Customer record count: 2000
[2025-07-21 10:04:39] [20250721_100439] ❌ Threshold condition not met: 2000 <= 5000

📈 Execution Summary:
               Metric  Value
  Pipeline Executions      3
Successful Executions      2
    Failed Executions      0
   Skipped Executions      1

✅ Task 1 completed successfully!


In [21]:
# Cell 1: Setup and Imports
import pandas as pd
import sqlite3
import json
import os
from datetime import datetime
import shutil

# Verify data exists
if not os.path.exists('/content/data/database.sqlite'):
    print("❌ Database not found! Please run 01_Data_Setup.ipynb first")
else:
    print("✅ Database found")


✅ Database found


In [22]:
# Cell 2: ForEach Pipeline Class
class ForEachPipeline:
    def __init__(self, db_path='/content/data/database.sqlite'):
        self.db_path = db_path
        self.log_file = '/content/logs/foreach_pipeline.log'
        self.execution_id = datetime.now().strftime('%Y%m%d_%H%M%S')

    def log(self, message):
        """Log pipeline execution"""
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        log_entry = f"[{timestamp}] [{self.execution_id}] {message}"
        print(log_entry)
        with open(self.log_file, 'a') as f:
            f.write(log_entry + '\n')

    def create_output_folder(self, table_name):
        """Create date-based folder structure for each table"""
        now = datetime.now()
        date_path = now.strftime('%Y/%m/%d')
        full_path = f'/content/output/{table_name}/{date_path}'
        os.makedirs(full_path, exist_ok=True)
        self.log(f"✅ Created folder structure: {full_path}")
        return full_path

    def copy_product_data(self, output_path):
        """Copy product data where productid > 100"""
        try:
            conn = sqlite3.connect(self.db_path)
            query = "SELECT * FROM Product WHERE ProductId > 100"
            product_data = pd.read_sql(query, conn)
            conn.close()

            # Create filename with timestamp
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f'product_data_{timestamp}.json'
            full_file_path = os.path.join(output_path, filename)

            # Save as JSON
            product_data.to_json(full_file_path, orient='records', indent=2)

            self.log(f"✅ Product data copied to: {full_file_path}")
            self.log(f"✅ Records copied: {len(product_data)}")

            return full_file_path, len(product_data)

        except Exception as e:
            self.log(f"❌ Error copying product data: {str(e)}")
            raise e

    def copy_customer_data(self, output_path):
        """Copy customer data where customer id > 100 and < 1000"""
        try:
            conn = sqlite3.connect(self.db_path)
            query = "SELECT * FROM Customer WHERE CustomerId > 100 AND CustomerId < 1000"
            customer_data = pd.read_sql(query, conn)
            conn.close()

            # Create filename with timestamp
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f'customer_data_{timestamp}.json'
            full_file_path = os.path.join(output_path, filename)

            # Save as JSON
            customer_data.to_json(full_file_path, orient='records', indent=2)

            self.log(f"✅ Customer data copied to: {full_file_path}")
            self.log(f"✅ Records copied: {len(customer_data)}")

            return full_file_path, len(customer_data)

        except Exception as e:
            self.log(f"❌ Error copying customer data: {str(e)}")
            raise e

    def execute_pipeline(self):
        """Execute the ForEach pipeline for both tables"""
        self.log("🚀 Starting ForEach Pipeline")

        results = {
            'pipeline_name': 'Foreach_Example2',
            'execution_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'tables_processed': []
        }

        try:
            # Define tables to process with their conditions
            tables_config = [
                {
                    'table_name': 'Product',
                    'condition': 'ProductId > 100',
                    'copy_function': self.copy_product_data
                },
                {
                    'table_name': 'Customer',
                    'condition': 'CustomerId > 100 AND CustomerId < 1000',
                    'copy_function': self.copy_customer_data
                }
            ]

            # Process each table (simulating ForEach activity)
            for table_config in tables_config:
                table_name = table_config['table_name']
                condition = table_config['condition']
                copy_function = table_config['copy_function']

                self.log(f"📋 Processing table: {table_name}")
                self.log(f"📋 Condition: {condition}")

                # Create output folder
                output_path = self.create_output_folder(table_name)

                # Copy data
                file_path, record_count = copy_function(output_path)

                # Store results
                table_result = {
                    'table_name': table_name,
                    'condition': condition,
                    'records_copied': record_count,
                    'output_file': file_path,
                    'status': 'SUCCESS'
                }
                results['tables_processed'].append(table_result)

                self.log(f"✅ Completed processing: {table_name}")

            results['status'] = 'SUCCESS'
            results['total_tables'] = len(tables_config)

            self.log("🎉 ForEach Pipeline completed successfully")

        except Exception as e:
            self.log(f"💥 Pipeline failed: {str(e)}")
            results['status'] = 'FAILED'
            results['error'] = str(e)

        return results


In [23]:
# Cell 3: Execute ForEach Pipeline
print("=== TASK 2: FOREACH PIPELINE ===")
print("Pipeline Name: Foreach_Example2")
print("Objective: Copy data using one copy activity with conditions")

foreach_pipeline = ForEachPipeline()
result = foreach_pipeline.execute_pipeline()


=== TASK 2: FOREACH PIPELINE ===
Pipeline Name: Foreach_Example2
Objective: Copy data using one copy activity with conditions
[2025-07-21 10:07:21] [20250721_100721] 🚀 Starting ForEach Pipeline
[2025-07-21 10:07:21] [20250721_100721] 📋 Processing table: Product
[2025-07-21 10:07:21] [20250721_100721] 📋 Condition: ProductId > 100
[2025-07-21 10:07:21] [20250721_100721] ✅ Created folder structure: /content/output/Product/2025/07/21
[2025-07-21 10:07:21] [20250721_100721] ✅ Product data copied to: /content/output/Product/2025/07/21/product_data_20250721_100721.json
[2025-07-21 10:07:21] [20250721_100721] ✅ Records copied: 100
[2025-07-21 10:07:21] [20250721_100721] ✅ Completed processing: Product
[2025-07-21 10:07:21] [20250721_100721] 📋 Processing table: Customer
[2025-07-21 10:07:21] [20250721_100721] 📋 Condition: CustomerId > 100 AND CustomerId < 1000
[2025-07-21 10:07:21] [20250721_100721] ✅ Created folder structure: /content/output/Customer/2025/07/21
[2025-07-21 10:07:21] [20250721_

In [24]:
# Cell 4: Display Results
print("\n=== FOREACH PIPELINE EXECUTION RESULTS ===")
print(json.dumps(result, indent=2))



=== FOREACH PIPELINE EXECUTION RESULTS ===
{
  "pipeline_name": "Foreach_Example2",
  "execution_time": "2025-07-21 10:07:21",
  "tables_processed": [
    {
      "table_name": "Product",
      "condition": "ProductId > 100",
      "records_copied": 100,
      "output_file": "/content/output/Product/2025/07/21/product_data_20250721_100721.json",
      "status": "SUCCESS"
    },
    {
      "table_name": "Customer",
      "condition": "CustomerId > 100 AND CustomerId < 1000",
      "records_copied": 899,
      "output_file": "/content/output/Customer/2025/07/21/customer_data_20250721_100721.json",
      "status": "SUCCESS"
    }
  ],
  "status": "SUCCESS",
  "total_tables": 2
}


In [25]:
# Cell 5: Validate Outputs
if result['status'] == 'SUCCESS':
    print("\n=== OUTPUT VALIDATION ===")

    for table_result in result['tables_processed']:
        table_name = table_result['table_name']
        output_file = table_result['output_file']

        print(f"\n--- {table_name} Table ---")

        if os.path.exists(output_file):
            print(f"✅ Output file exists: {output_file}")

            # Check file size
            file_size = os.path.getsize(output_file)
            print(f"✅ File size: {file_size} bytes")

            # Load and display sample data
            with open(output_file, 'r') as f:
                sample_data = json.load(f)

            print(f"✅ Records in file: {len(sample_data)}")

            if sample_data:
                print("Sample records:")
                for i, record in enumerate(sample_data[:2]):
                    print(f"  Record {i+1}: {record}")
        else:
            print(f"❌ Output file not found: {output_file}")



=== OUTPUT VALIDATION ===

--- Product Table ---
✅ Output file exists: /content/output/Product/2025/07/21/product_data_20250721_100721.json
✅ File size: 20108 bytes
✅ Records in file: 100
Sample records:
  Record 1: {'ProductId': 101, 'ProductName': 'Product_101', 'Category': 'Sports', 'Price': 504.2, 'StockQuantity': 61, 'Supplier': 'Supplier_12', 'LaunchDate': '2020-12-06 00:00:00'}
  Record 2: {'ProductId': 102, 'ProductName': 'Product_102', 'Category': 'Electronics', 'Price': 576.28, 'StockQuantity': 83, 'Supplier': 'Supplier_5', 'LaunchDate': '2020-12-13 00:00:00'}

--- Customer Table ---
✅ Output file exists: /content/output/Customer/2025/07/21/customer_data_20250721_100721.json
✅ File size: 203622 bytes
✅ Records in file: 899
Sample records:
  Record 1: {'CustomerId': 101, 'CustomerName': 'Customer_0101', 'Email': 'customer101@email.com', 'Phone': '+1-153-219-2648', 'RegistrationDate': '2020-04-10 00:00:00', 'Status': 'Active', 'CreditScore': 549}
  Record 2: {'CustomerId': 102

In [26]:
# Cell 6: Data Quality Checks
def perform_data_quality_checks():
    """Perform data quality checks on copied data"""
    print("\n=== DATA QUALITY CHECKS ===")

    conn = sqlite3.connect('/content/data/database.sqlite')

    # Check Product data
    product_query = "SELECT COUNT(*) as count FROM Product WHERE ProductId > 100"
    product_count = pd.read_sql(product_query, conn).iloc[0]['count']
    print(f"✅ Products with ID > 100: {product_count}")

    # Check Customer data
    customer_query = "SELECT COUNT(*) as count FROM Customer WHERE CustomerId > 100 AND CustomerId < 1000"
    customer_count = pd.read_sql(customer_query, conn).iloc[0]['count']
    print(f"✅ Customers with ID > 100 and < 1000: {customer_count}")

    # Show sample data from source
    print("\n--- Source Data Samples ---")
    sample_products = pd.read_sql("SELECT * FROM Product WHERE ProductId > 100 LIMIT 3", conn)
    print("Sample Products:")
    print(sample_products.to_string(index=False))

    sample_customers = pd.read_sql("SELECT * FROM Customer WHERE CustomerId > 100 AND CustomerId < 1000 LIMIT 3", conn)
    print("\nSample Customers:")
    print(sample_customers.to_string(index=False))

    conn.close()

perform_data_quality_checks()



=== DATA QUALITY CHECKS ===
✅ Products with ID > 100: 100
✅ Customers with ID > 100 and < 1000: 899

--- Source Data Samples ---
Sample Products:
 ProductId ProductName    Category  Price  StockQuantity    Supplier          LaunchDate
       101 Product_101      Sports 504.20             61 Supplier_12 2020-12-06 00:00:00
       102 Product_102 Electronics 576.28             83  Supplier_5 2020-12-13 00:00:00
       103 Product_103 Electronics 770.87             88 Supplier_17 2020-12-20 00:00:00

Sample Customers:
 CustomerId  CustomerName                 Email           Phone    RegistrationDate Status  CreditScore
        101 Customer_0101 customer101@email.com +1-153-219-2648 2020-04-10 00:00:00 Active          549
        102 Customer_0102 customer102@email.com +1-519-521-5199 2020-04-11 00:00:00 Active          539
        103 Customer_0103 customer103@email.com +1-951-353-3557 2020-04-12 00:00:00 Active          679


In [27]:
# Cell 7: Folder Structure Visualization
def show_foreach_folder_structure():
    """Display the created folder structure"""
    print("\n=== FOREACH PIPELINE FOLDER STRUCTURE ===")

    def print_tree(directory, prefix="", max_depth=4, current_depth=0):
        if current_depth > max_depth:
            return

        items = []
        try:
            items = sorted(os.listdir(directory))
        except (PermissionError, FileNotFoundError):
            return

        for i, item in enumerate(items):
            if item.startswith('.'):
                continue

            item_path = os.path.join(directory, item)
            is_last = i == len(items) - 1
            current_prefix = "└── " if is_last else "├── "

            if os.path.isfile(item_path):
                file_size = os.path.getsize(item_path)
                print(f"{prefix}{current_prefix}{item} ({file_size} bytes)")
            else:
                print(f"{prefix}{current_prefix}{item}/")

            if os.path.isdir(item_path):
                next_prefix = prefix + ("    " if is_last else "│   ")
                print_tree(item_path, next_prefix, max_depth, current_depth + 1)

    print_tree("/content/output")

show_foreach_folder_structure()

print("\n✅ Task 2 (ForEach Pipeline) completed successfully!")



=== FOREACH PIPELINE FOLDER STRUCTURE ===
├── Customer/
│   └── 2025/
│       └── 07/
│           └── 21/
│               ├── customer_data_20250721_100255.json (454775 bytes)
│               ├── customer_data_20250721_100439.json (454775 bytes)
│               └── customer_data_20250721_100721.json (203622 bytes)
├── CustomerWithAddress/
└── Product/
    └── 2025/
        └── 07/
            └── 21/
                └── product_data_20250721_100721.json (20108 bytes)

✅ Task 2 (ForEach Pipeline) completed successfully!


In [28]:
# Cell 1: Setup and Imports
import pandas as pd
import sqlite3
import json
import os
from datetime import datetime
import pyarrow as pa
import pyarrow.parquet as pq

# Verify data exists
required_files = [
    '/content/data/database.sqlite',
    '/content/data/customer_address.csv'
]

for file_path in required_files:
    if not os.path.exists(file_path):
        print(f"❌ {file_path} not found! Please run 01_Data_Setup.ipynb first")
    else:
        print(f"✅ {file_path} found")


✅ /content/data/database.sqlite found
✅ /content/data/customer_address.csv found


In [29]:
# Cell 2: Join Pipeline Class
class JoinPipeline:
    def __init__(self, db_path='/content/data/database.sqlite'):
        self.db_path = db_path
        self.csv_path = '/content/data/customer_address.csv'
        self.log_file = '/content/logs/join_pipeline.log'
        self.execution_id = datetime.now().strftime('%Y%m%d_%H%M%S')

    def log(self, message):
        """Log pipeline execution"""
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        log_entry = f"[{timestamp}] [{self.execution_id}] {message}"
        print(log_entry)
        with open(self.log_file, 'a') as f:
            f.write(log_entry + '\n')

    def create_output_folder(self):
        """Create output folder for joined data"""
        now = datetime.now()
        date_path = now.strftime('%Y/%m/%d')
        full_path = f'/content/output/CustomerWithAddress/{date_path}'
        os.makedirs(full_path, exist_ok=True)
        self.log(f"✅ Created folder structure: {full_path}")
        return full_path

    def read_customer_data(self):
        """Read customer data from SQL database"""
        try:
            conn = sqlite3.connect(self.db_path)
            query = "SELECT * FROM Customer"
            customer_data = pd.read_sql(query, conn)
            conn.close()

            self.log(f"✅ Customer data loaded: {len(customer_data)} records")
            return customer_data

        except Exception as e:
            self.log(f"❌ Error reading customer data: {str(e)}")
            raise e

    def read_address_data(self):
        """Read customer address data from CSV"""
        try:
            address_data = pd.read_csv(self.csv_path)

            # Clean column names (remove whitespace)
            address_data.columns = address_data.columns.str.strip()

            self.log(f"✅ Address data loaded: {len(address_data)} records")
            return address_data

        except Exception as e:
            self.log(f"❌ Error reading address data: {str(e)}")
            raise e

    def join_data(self, customer_data, address_data):
        """Join customer and address data"""
        try:
            # Perform inner join on CustomerId
            joined_data = pd.merge(
                customer_data,
                address_data,
                on='CustomerId',
                how='inner'
            )

            self.log(f"✅ Data joined successfully: {len(joined_data)} records")

            # Apply filter: CustomerId > 1000 AND CustomerId < 2000
            filtered_data = joined_data[
                (joined_data['CustomerId'] > 1000) &
                (joined_data['CustomerId'] < 2000)
            ]

            self.log(f"✅ Data filtered (1000 < CustomerId < 2000): {len(filtered_data)} records")

            # Sort by CustomerId in ascending order
            sorted_data = filtered_data.sort_values('CustomerId', ascending=True)

            self.log(f"✅ Data sorted by CustomerId (ascending)")

            return sorted_data

        except Exception as e:
            self.log(f"❌ Error joining data: {str(e)}")
            raise e

    def save_as_parquet(self, data, output_path):
        """Save joined data as Parquet file"""
        try:
            # Create filename with timestamp
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f'customer_with_address_{timestamp}.parquet'
            full_file_path = os.path.join(output_path, filename)

            # Save as Parquet
            data.to_parquet(full_file_path, index=False)

            self.log(f"✅ Data saved as Parquet: {full_file_path}")
            self.log(f"✅ Records saved: {len(data)}")

            return full_file_path, len(data)

        except Exception as e:
            self.log(f"❌ Error saving Parquet file: {str(e)}")
            raise e

    def execute_pipeline(self):
        """Execute the complete join pipeline"""
        self.log("🚀 Starting Join Pipeline")

        try:
            # Step 1: Read customer data from SQL
            customer_data = self.read_customer_data()

            # Step 2: Read address data from CSV
            address_data = self.read_address_data()

            # Step 3: Join data
            joined_data = self.join_data(customer_data, address_data)

            # Step 4: Create output folder
            output_path = self.create_output_folder()

            # Step 5: Save as Parquet
            file_path, saved_records = self.save_as_parquet(joined_data, output_path)

            result = {
                'status': 'SUCCESS',
                'customer_records': len(customer_data),
                'address_records': len(address_data),
                'joined_records': len(joined_data),
                'saved_records': saved_records,
                'output_file': file_path,
                'execution_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            self.log("🎉 Join Pipeline completed successfully")

            return result

        except Exception as e:
            self.log(f"💥 Pipeline failed: {str(e)}")
            return {
                'status': 'FAILED',
                'error': str(e),
                'execution_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }


In [30]:
# Cell 3: Execute Join Pipeline
print("=== TASK 3: JOIN PIPELINE ===")
print("Objective: Join Customer and Address data, filter and save as Parquet")

join_pipeline = JoinPipeline()
result = join_pipeline.execute_pipeline()


=== TASK 3: JOIN PIPELINE ===
Objective: Join Customer and Address data, filter and save as Parquet
[2025-07-21 10:09:54] [20250721_100954] 🚀 Starting Join Pipeline
[2025-07-21 10:09:54] [20250721_100954] ✅ Customer data loaded: 2000 records
[2025-07-21 10:09:54] [20250721_100954] ✅ Address data loaded: 2000 records
[2025-07-21 10:09:54] [20250721_100954] ✅ Data joined successfully: 2000 records
[2025-07-21 10:09:54] [20250721_100954] ✅ Data filtered (1000 < CustomerId < 2000): 999 records
[2025-07-21 10:09:54] [20250721_100954] ✅ Data sorted by CustomerId (ascending)
[2025-07-21 10:09:54] [20250721_100954] ✅ Created folder structure: /content/output/CustomerWithAddress/2025/07/21
[2025-07-21 10:09:54] [20250721_100954] ✅ Data saved as Parquet: /content/output/CustomerWithAddress/2025/07/21/customer_with_address_20250721_100954.parquet
[2025-07-21 10:09:54] [20250721_100954] ✅ Records saved: 999
[2025-07-21 10:09:54] [20250721_100954] 🎉 Join Pipeline completed successfully


In [31]:
# Cell 4: Display Results
print("\n=== JOIN PIPELINE EXECUTION RESULTS ===")
print(json.dumps(result, indent=2))



=== JOIN PIPELINE EXECUTION RESULTS ===
{
  "status": "SUCCESS",
  "customer_records": 2000,
  "address_records": 2000,
  "joined_records": 999,
  "saved_records": 999,
  "output_file": "/content/output/CustomerWithAddress/2025/07/21/customer_with_address_20250721_100954.parquet",
  "execution_time": "2025-07-21 10:09:54"
}


In [32]:
# Cell 5: Validate Output
if result['status'] == 'SUCCESS':
    print("\n=== OUTPUT VALIDATION ===")

    output_file = result['output_file']

    if os.path.exists(output_file):
        print(f"✅ Output file exists: {output_file}")

        # Check file size
        file_size = os.path.getsize(output_file)
        print(f"✅ File size: {file_size} bytes")

        # Load and display sample data
        parquet_data = pd.read_parquet(output_file)
        print(f"✅ Records in Parquet file: {len(parquet_data)}")

        # Show data structure
        print(f"✅ Columns: {list(parquet_data.columns)}")

        # Show sample records
        print("\n--- Sample Records ---")
        print(parquet_data.head().to_string(index=False))

        # Verify sorting
        print("\n--- Sorting Verification ---")
        print(f"Min CustomerId: {parquet_data['CustomerId'].min()}")
        print(f"Max CustomerId: {parquet_data['CustomerId'].max()}")

        # Check if data is sorted
        is_sorted = parquet_data['CustomerId'].is_monotonic_increasing
        print(f"✅ Data is sorted (ascending): {is_sorted}")

    else:
        print(f"❌ Output file not found: {output_file}")



=== OUTPUT VALIDATION ===
✅ Output file exists: /content/output/CustomerWithAddress/2025/07/21/customer_with_address_20250721_100954.parquet
✅ File size: 61664 bytes
✅ Records in Parquet file: 999
✅ Columns: ['CustomerId', 'CustomerName', 'Email', 'Phone', 'RegistrationDate', 'Status', 'CreditScore', 'Address', 'City', 'State', 'ZipCode', 'Country']

--- Sample Records ---
 CustomerId  CustomerName                  Email           Phone    RegistrationDate   Status  CreditScore        Address         City State  ZipCode Country
       1001 Customer_1001 customer1001@email.com +1-475-843-2749 2022-09-27 00:00:00 Inactive          620   5190 Main St Philadelphia    TX    28263     USA
       1002 Customer_1002 customer1002@email.com +1-727-703-1781 2022-09-28 00:00:00   Active          623 708 Second Ave       Dallas    TX    63260     USA
       1003 Customer_1003 customer1003@email.com +1-480-779-3105 2022-09-29 00:00:00   Active          683  6705 First St     San Jose    PA    53306

In [33]:
# Cell 6: Data Quality and Join Analysis
def perform_join_analysis():
    """Perform detailed analysis of the join operation"""
    print("\n=== JOIN ANALYSIS ===")

    # Read source data
    conn = sqlite3.connect('/content/data/database.sqlite')
    customers = pd.read_sql("SELECT * FROM Customer", conn)
    conn.close()

    addresses = pd.read_csv('/content/data/customer_address.csv')
    addresses.columns = addresses.columns.str.strip()

    print(f"📊 Source Data Summary:")
    print(f"  - Customer records: {len(customers)}")
    print(f"  - Address records: {len(addresses)}")

    # Analyze join keys
    customer_ids = set(customers['CustomerId'])
    address_ids = set(addresses['CustomerId'])

    common_ids = customer_ids.intersection(address_ids)
    customer_only = customer_ids - address_ids
    address_only = address_ids - customer_ids

    print(f"\n📊 Join Key Analysis:")
    print(f"  - Common CustomerIds: {len(common_ids)}")
    print(f"  - Customer-only IDs: {len(customer_only)}")
    print(f"  - Address-only IDs: {len(address_only)}")

    # Filter analysis
    filtered_range = [id for id in common_ids if 1000 < id < 2000]
    print(f"  - IDs in range (1000 < ID < 2000): {len(filtered_range)}")

    # Show join result breakdown
    joined_full = pd.merge(customers, addresses, on='CustomerId', how='inner')
    print(f"\n📊 Join Results:")
    print(f"  - Total joined records: {len(joined_full)}")
    print(f"  - Records after filtering: {len(filtered_range)}")
    print(f"  - Join success rate: {(len(common_ids)/len(customers)*100):.1f}%")

perform_join_analysis()



=== JOIN ANALYSIS ===
📊 Source Data Summary:
  - Customer records: 2000
  - Address records: 2000

📊 Join Key Analysis:
  - Common CustomerIds: 2000
  - Customer-only IDs: 0
  - Address-only IDs: 0
  - IDs in range (1000 < ID < 2000): 999

📊 Join Results:
  - Total joined records: 2000
  - Records after filtering: 999
  - Join success rate: 100.0%


In [34]:
# Cell 7: Parquet File Analysis
def analyze_parquet_file():
    """Analyze the structure and content of the Parquet file"""
    print("\n=== PARQUET FILE ANALYSIS ===")

    if result['status'] == 'SUCCESS':
        parquet_file = result['output_file']

        # Read parquet file
        df = pd.read_parquet(parquet_file)

        print(f"📄 File: {os.path.basename(parquet_file)}")
        print(f"📊 Shape: {df.shape}")
        print(f"📊 Memory usage: {df.memory_usage(deep=True).sum() / 1024:.1f} KB")

        # Column analysis
        print(f"\n📊 Column Information:")
        for col in df.columns:
            dtype = df[col].dtype
            null_count = df[col].isnull().sum()
            print(f"  - {col}: {dtype} ({null_count} nulls)")

        # Data distribution
        print(f"\n📊 CustomerId Distribution:")
        print(f"  - Min: {df['CustomerId'].min()}")
        print(f"  - Max: {df['CustomerId'].max()}")
        print(f"  - Range: {df['CustomerId'].max() - df['CustomerId'].min()}")

        # Geographic distribution
        if 'City' in df.columns:
            city_counts = df['City'].value_counts().head()
            print(f"\n📊 Top Cities:")
            for city, count in city_counts.items():
                print(f"  - {city}: {count} customers")

analyze_parquet_file()

print("\n✅ Task 3 (Join Pipeline) completed successfully!")



=== PARQUET FILE ANALYSIS ===
📄 File: customer_with_address_20250721_100954.parquet
📊 Shape: (999, 12)
📊 Memory usage: 623.5 KB

📊 Column Information:
  - CustomerId: int64 (0 nulls)
  - CustomerName: object (0 nulls)
  - Email: object (0 nulls)
  - Phone: object (0 nulls)
  - RegistrationDate: object (0 nulls)
  - Status: object (0 nulls)
  - CreditScore: int64 (0 nulls)
  - Address: object (0 nulls)
  - City: object (0 nulls)
  - State: object (0 nulls)
  - ZipCode: int64 (0 nulls)
  - Country: object (0 nulls)

📊 CustomerId Distribution:
  - Min: 1001
  - Max: 1999
  - Range: 998

📊 Top Cities:
  - San Antonio: 124 customers
  - Philadelphia: 117 customers
  - Dallas: 105 customers
  - Chicago: 102 customers
  - Los Angeles: 101 customers

✅ Task 3 (Join Pipeline) completed successfully!


In [35]:
# Cell 1: Setup and Imports
import pandas as pd
import sqlite3
import json
import os
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np


In [36]:
# Cell 2: Load All Pipeline Results
def load_pipeline_results():
    """Load results from all executed pipelines"""
    results = {
        'threshold_pipeline': None,
        'foreach_pipeline': None,
        'join_pipeline': None,
        'execution_summary': {}
    }

    # Check log files
    log_files = [
        '/content/logs/threshold_pipeline.log',
        '/content/logs/foreach_pipeline.log',
        '/content/logs/join_pipeline.log'
    ]

    for log_file in log_files:
        if os.path.exists(log_file):
            pipeline_name = os.path.basename(log_file).replace('.log', '')
            with open(log_file, 'r') as f:
                logs = f.readlines()
            results['execution_summary'][pipeline_name] = {
                'executed': True,
                'log_entries': len(logs),
                'last_execution': logs[-1].split(']')[0].strip('[') if logs else 'Unknown'
            }
        else:
            pipeline_name = os.path.basename(log_file).replace('.log', '')
            results['execution_summary'][pipeline_name] = {
                'executed': False,
                'log_entries': 0,
                'last_execution': 'Not executed'
            }

    return results

pipeline_results = load_pipeline_results()


In [37]:
# Cell 3: Create Executive Summary
def create_executive_summary():
    """Create executive summary dashboard"""
    print("📊 EXECUTIVE SUMMARY")
    print("=" * 50)

    # Count output files
    output_files = []
    for root, dirs, files in os.walk('/content/output'):
        for file in files:
            if not file.startswith('.'):
                output_files.append(os.path.join(root, file))

    print(f"📈 Total Output Files Generated: {len(output_files)}")

    # Pipeline execution summary
    executed_pipelines = sum(1 for p in pipeline_results['execution_summary'].values() if p['executed'])
    total_pipelines = len(pipeline_results['execution_summary'])

    print(f"📈 Pipelines Executed: {executed_pipelines}/{total_pipelines}")

    # File format breakdown
    file_formats = {}
    for file_path in output_files:
        ext = os.path.splitext(file_path)[1].lower()
        file_formats[ext] = file_formats.get(ext, 0) + 1

    print(f"📈 File Formats:")
    for fmt, count in file_formats.items():
        print(f"  - {fmt}: {count} files")

    # Total data processed
    total_size = sum(os.path.getsize(f) for f in output_files)
    print(f"📈 Total Data Processed: {total_size/1024:.1f} KB")

    return {
        'total_files': len(output_files),
        'executed_pipelines': executed_pipelines,
        'total_pipelines': total_pipelines,
        'file_formats': file_formats,
        'total_size_kb': total_size/1024
    }

summary_stats = create_executive_summary()


📊 EXECUTIVE SUMMARY
📈 Total Output Files Generated: 5
📈 Pipelines Executed: 3/3
📈 File Formats:
  - .parquet: 1 files
  - .json: 4 files
📈 Total Data Processed: 1166.9 KB


In [38]:
# Cell 4: Pipeline Performance Dashboard
def create_pipeline_dashboard():
    """Create pipeline performance visualizations"""
    print("\n📊 PIPELINE PERFORMANCE DASHBOARD")
    print("=" * 50)

    # Pipeline execution status
    pipeline_names = list(pipeline_results['execution_summary'].keys())
    execution_status = [pipeline_results['execution_summary'][p]['executed'] for p in pipeline_names]

    # Create pipeline status chart
    fig1 = go.Figure(data=[
        go.Bar(
            name='Executed',
            x=pipeline_names,
            y=[1 if status else 0 for status in execution_status],
            marker_color=['green' if status else 'red' for status in execution_status]
        )
    ])

    fig1.update_layout(
        title='Pipeline Execution Status',
        xaxis_title='Pipeline',
        yaxis_title='Status (1=Executed, 0=Not Executed)',
        showlegend=False
    )

    fig1.show()

    # Log entries chart
    log_counts = [pipeline_results['execution_summary'][p]['log_entries'] for p in pipeline_names]

    fig2 = go.Figure(data=[
        go.Bar(
            name='Log Entries',
            x=pipeline_names,
            y=log_counts,
            marker_color='blue'
        )
    ])

    fig2.update_layout(
        title='Pipeline Log Entries',
        xaxis_title='Pipeline',
        yaxis_title='Number of Log Entries'
    )

    fig2.show()

create_pipeline_dashboard()



📊 PIPELINE PERFORMANCE DASHBOARD


In [39]:
# Cell 5: Data Quality Dashboard (continued)
def create_data_quality_dashboard():
    """Create data quality analysis dashboard"""

    print("\n🔍 DATA QUALITY DASHBOARD")
    print("="*50)

    # Load source data for analysis
    conn = sqlite3.connect('/content/data/database.sqlite')
    customers = pd.read_sql("SELECT * FROM Customer", conn)
    products = pd.read_sql("SELECT * FROM Product", conn)
    conn.close()

    addresses = pd.read_csv('/content/data/customer_address.csv')

    # Data completeness analysis
    print("\n📊 Data Completeness Analysis:")

    # Customer data quality
    customer_nulls = customers.isnull().sum()
    print("\n🔹 Customer Data (Total: {len(customers)} records):")
    for col, null_count in customer_nulls.items():
        completeness = (len(customers) - null_count)/len(customers) * 100
        print(f" - {col}: {completeness:.1f}% complete")

    # Product data quality
    product_nulls = products.isnull().sum()
    print("\n🔹 Product Data (Total: {len(products)} records):")
    for col, null_count in product_nulls.items():
        completeness = (len(products) - null_count)/len(products) * 100
        print(f" - {col}: {completeness:.1f}% complete")

    # Address data quality
    address_nulls = addresses.isnull().sum()
    print("\n🔹 Address Data (Total: {len(addresses)} records):")
    for col, null_count in address_nulls.items():
        completeness = (len(addresses) - null_count)/len(addresses) * 100
        print(f" - {col}: {completeness:.1f}% complete")

    # Create data quality visualization
    tables = ['Customer', 'Product', 'Address']
    completeness_data = []

    for table, df in zip(tables, [customers, products, addresses]):
        for col in df.columns:
            null_count = df[col].isnull().sum()
            completeness = (len(df) - null_count)/len(df) * 100
            completeness_data.append({
                'table': table,
                'column': col,
                'completeness': completeness
            })

    completeness_df = pd.DataFrame(completeness_data)

    fig = px.bar(completeness_df,
                 x='column', y='completeness',
                 color='table',
                 barmode='group',
                 title='Data Completeness by Table and Column',
                 labels={'completeness': 'Completeness (%)', 'column': 'Column'},
                 height=500)
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

create_data_quality_dashboard()



🔍 DATA QUALITY DASHBOARD

📊 Data Completeness Analysis:

🔹 Customer Data (Total: {len(customers)} records):
 - CustomerId: 100.0% complete
 - CustomerName: 100.0% complete
 - Email: 100.0% complete
 - Phone: 100.0% complete
 - RegistrationDate: 100.0% complete
 - Status: 100.0% complete
 - CreditScore: 100.0% complete

🔹 Product Data (Total: {len(products)} records):
 - ProductId: 100.0% complete
 - ProductName: 100.0% complete
 - Category: 100.0% complete
 - Price: 100.0% complete
 - StockQuantity: 100.0% complete
 - Supplier: 100.0% complete
 - LaunchDate: 100.0% complete

🔹 Address Data (Total: {len(addresses)} records):
 - CustomerId: 100.0% complete
 - Address: 100.0% complete
 - City: 100.0% complete
 - State: 100.0% complete
 - ZipCode: 100.0% complete
 - Country: 100.0% complete


In [40]:
# Cell 6: Output Data Analysis
def analyze_output_data():
    """Analyze all output files generated by pipelines"""

    print("\n📂 OUTPUT DATA ANALYSIS")
    print("="*50)

    # Find all output files
    output_files = []
    for root, dirs, files in os.walk('/content/output'):
        for file in files:
            if not file.startswith('.'):
                output_files.append(os.path.join(root, file))

    print(f"Found {len(output_files)} output files")

    # Analyze each output file
    output_stats = []

    for file_path in output_files:
        file_name = os.path.basename(file_path)
        file_size = os.path.getsize(file_path)
        file_type = os.path.splitext(file_path)[1].lower()
        record_count = 0

        try:
            if file_type == '.json':
                with open(file_path, 'r') as f:
                    data = json.load(f)
                record_count = len(data)
            elif file_type == '.parquet':
                data = pd.read_parquet(file_path)
                record_count = len(data)

            output_stats.append({
                'file_name': file_name,
                'file_type': file_type,
                'file_size_kb': file_size/1024,
                'record_count': record_count,
                'path': os.path.dirname(file_path)
            })
        except Exception as e:
            print(f"⚠️ Error analyzing {file_name}: {str(e)}")

    if output_stats:
        stats_df = pd.DataFrame(output_stats)

        # Show summary
        print("\n📊 Output File Summary:")
        print(stats_df.groupby('file_type').agg({
            'file_name': 'count',
            'file_size_kb': 'sum',
            'record_count': 'sum'
        }).rename(columns={
            'file_name': 'file_count',
            'file_size_kb': 'total_size_kb',
            'record_count': 'total_records'
        }))

        # Create visualizations
        fig1 = px.pie(stats_df,
                     names='file_type',
                     values='file_size_kb',
                     title='Output Data Distribution by File Type (Size)')
        fig1.show()

        fig2 = px.bar(stats_df,
                     x='path',
                     y='record_count',
                     color='file_type',
                     title='Record Count by Output Folder',
                     labels={'path': 'Output Path', 'record_count': 'Record Count'})
        fig2.update_layout(xaxis_tickangle=-45)
        fig2.show()

        return stats_df
    else:
        print("⚠️ No valid output files found for analysis")
        return None

output_stats = analyze_output_data()



📂 OUTPUT DATA ANALYSIS
Found 5 output files

📊 Output File Summary:
           file_count  total_size_kb  total_records
file_type                                          
.json               4     1106.71875           4999
.parquet            1       60.21875            999


In [41]:
# Cell 7: Threshold Pipeline Analysis
def analyze_threshold_pipeline():
    """Analyze threshold pipeline performance"""

    print("\n📈 THRESHOLD PIPELINE ANALYSIS")
    print("="*50)

    # Check if threshold pipeline log exists
    log_file = '/content/logs/threshold_pipeline.log'
    if not os.path.exists(log_file):
        print("⚠️ Threshold pipeline log not found")
        return

    # Parse log file
    with open(log_file, 'r') as f:
        logs = f.readlines()

    # Extract execution results
    executions = []
    current_exec = {}

    for log in logs:
        if 'Starting Threshold Pipeline' in log:
            if current_exec:
                executions.append(current_exec)
            current_exec = {'id': log.split('[')[1].split(']')[0]}
        elif 'Threshold condition met' in log:
            current_exec['condition'] = 'met'
        elif 'Threshold condition not met' in log:
            current_exec['condition'] = 'not met'
        elif 'Records copied' in log:
            current_exec['records_copied'] = int(log.split(':')[-1].strip())
        elif 'Pipeline completed successfully' in log:
            current_exec['status'] = 'success'
        elif 'Pipeline failed' in log:
            current_exec['status'] = 'failed'

    if current_exec:
        executions.append(current_exec)

    if not executions:
        print("⚠️ No pipeline executions found in log")
        return

    # Create analysis
    exec_df = pd.DataFrame(executions)
    print(f"\nFound {len(exec_df)} pipeline executions:")
    print(exec_df)

    # Visualizations
    if 'condition' in exec_df.columns:
        fig1 = px.pie(exec_df,
                     names='condition',
                     title='Threshold Condition Outcomes')
        fig1.show()

    if 'records_copied' in exec_df.columns:
        fig2 = px.bar(exec_df[exec_df['records_copied'] > 0],
                     y='records_copied',
                     title='Records Copied in Successful Executions')
        fig2.show()

analyze_threshold_pipeline()



📈 THRESHOLD PIPELINE ANALYSIS

Found 3 pipeline executions:
                    id condition  records_copied   status
0  2025-07-21 10:02:55       met          2000.0  success
1  2025-07-21 10:04:39       met          2000.0  success
2  2025-07-21 10:04:39   not met             NaN      NaN


In [42]:
# Cell 8: Join Pipeline Analysis
def analyze_join_pipeline():
    """Analyze join pipeline performance and results"""

    print("\n🤝 JOIN PIPELINE ANALYSIS")
    print("="*50)

    # Find latest parquet output
    parquet_files = []
    for root, dirs, files in os.walk('/content/output/CustomerWithAddress'):
        for file in files:
            if file.endswith('.parquet'):
                parquet_files.append(os.path.join(root, file))

    if not parquet_files:
        print("⚠️ No parquet output files found")
        return

    # Use most recent file
    latest_file = max(parquet_files, key=os.path.getctime)
    print(f"Analyzing latest output file: {latest_file}")

    # Load the data
    joined_data = pd.read_parquet(latest_file)

    # Analyze the joined data
    print(f"\n🔹 Joined Data Shape: {joined_data.shape}")
    print(f"🔹 Columns: {list(joined_data.columns)}")

    # Show data distribution
    fig1 = px.histogram(joined_data,
                       x='CustomerId',
                       title='Distribution of Customer IDs in Joined Data')
    fig1.show()

    if 'City' in joined_data.columns:
        fig2 = px.bar(joined_data['City'].value_counts().head(10),
                     title='Top 10 Cities in Joined Data',
                     labels={'value': 'Count', 'index': 'City'})
        fig2.show()

    if 'Status' in joined_data.columns:
        fig3 = px.pie(joined_data,
                     names='Status',
                     title='Customer Status Distribution in Joined Data')
        fig3.show()

    # Show sample data
    print("\nSample of Joined Data:")
    print(joined_data.head().to_string(index=False))

analyze_join_pipeline()



🤝 JOIN PIPELINE ANALYSIS
Analyzing latest output file: /content/output/CustomerWithAddress/2025/07/21/customer_with_address_20250721_100954.parquet

🔹 Joined Data Shape: (999, 12)
🔹 Columns: ['CustomerId', 'CustomerName', 'Email', 'Phone', 'RegistrationDate', 'Status', 'CreditScore', 'Address', 'City', 'State', 'ZipCode', 'Country']



Sample of Joined Data:
 CustomerId  CustomerName                  Email           Phone    RegistrationDate   Status  CreditScore        Address         City State  ZipCode Country
       1001 Customer_1001 customer1001@email.com +1-475-843-2749 2022-09-27 00:00:00 Inactive          620   5190 Main St Philadelphia    TX    28263     USA
       1002 Customer_1002 customer1002@email.com +1-727-703-1781 2022-09-28 00:00:00   Active          623 708 Second Ave       Dallas    TX    63260     USA
       1003 Customer_1003 customer1003@email.com +1-480-779-3105 2022-09-29 00:00:00   Active          683  6705 First St     San Jose    PA    53306     USA
       1004 Customer_1004 customer1004@email.com +1-895-125-5976 2022-09-30 00:00:00   Active          645  3714 First St     San Jose    IL    68418     USA
       1005 Customer_1005 customer1005@email.com +1-546-165-6879 2022-10-01 00:00:00 Inactive          402   2476 Park Rd    San Diego    PA    10148     USA


In [43]:
# Cell 9: Create Final Summary Report
def create_final_report():
    """Generate final summary report of all pipelines"""

    print("\n📑 FINAL PROJECT SUMMARY REPORT")
    print("="*50)
    print(f"Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # 1. Pipeline Execution Summary
    print("1. PIPELINE EXECUTION SUMMARY")
    print("-"*50)

    pipeline_status = []
    for pipeline in ['threshold_pipeline', 'foreach_pipeline', 'join_pipeline']:
        log_file = f'/content/logs/{pipeline}.log'
        if os.path.exists(log_file):
            with open(log_file, 'r') as f:
                logs = f.readlines()
            last_exec = logs[-1] if logs else "No executions found"
            status = "Executed" if logs else "Not executed"
        else:
            status = "Not executed"
            last_exec = "No log file found"

        pipeline_status.append({
            'Pipeline': pipeline.replace('_', ' ').title(),
            'Status': status,
            'Last Execution': last_exec.split(']')[0].strip('[') if ']' in last_exec else last_exec
        })

    print(pd.DataFrame(pipeline_status).to_string(index=False))

    # 2. Data Processing Summary
    print("\n2. DATA PROCESSING SUMMARY")
    print("-"*50)

    # Count records in source tables
    conn = sqlite3.connect('/content/data/database.sqlite')
    customer_count = pd.read_sql("SELECT COUNT(*) FROM Customer", conn).iloc[0,0]
    product_count = pd.read_sql("SELECT COUNT(*) FROM Product", conn).iloc[0,0]
    conn.close()

    address_count = len(pd.read_csv('/content/data/customer_address.csv'))

    print(f"🔹 Source Data Records:")
    print(f" - Customers: {customer_count}")
    print(f" - Products: {product_count}")
    print(f" - Addresses: {address_count}")

    # Count output records if available
    if output_stats is not None:
        total_output_records = output_stats['record_count'].sum()
        print(f"\n🔹 Output Data Records: {total_output_records}")

    # 3. Storage Usage
    print("\n3. STORAGE USAGE")
    print("-"*50)

    def get_dir_size(path):
        total = 0
        for dirpath, dirnames, filenames in os.walk(path):
            for f in filenames:
                fp = os.path.join(dirpath, f)
                total += os.path.getsize(fp)
        return total

    output_size = get_dir_size('/content/output')/1024  # KB
    data_size = get_dir_size('/content/data')/1024  # KB

    print(f"🔹 Source Data Size: {data_size:.1f} KB")
    print(f"🔹 Output Data Size: {output_size:.1f} KB")

    # 4. Recommendations
    print("\n4. RECOMMENDATIONS & NEXT STEPS")
    print("-"*50)
    print("🔹 Consider adding more data validation checks in pipelines")
    print("🔹 Implement automated testing for pipeline components")
    print("🔹 Add monitoring for data quality metrics over time")
    print("🔹 Consider partitioning output data for large datasets")
    print("🔹 Implement error notification system for failed pipelines")

create_final_report()



📑 FINAL PROJECT SUMMARY REPORT
Report generated: 2025-07-21 10:16:21

1. PIPELINE EXECUTION SUMMARY
--------------------------------------------------
          Pipeline   Status      Last Execution
Threshold Pipeline Executed 2025-07-21 10:04:39
  Foreach Pipeline Executed 2025-07-21 10:07:21
     Join Pipeline Executed 2025-07-21 10:09:54

2. DATA PROCESSING SUMMARY
--------------------------------------------------
🔹 Source Data Records:
 - Customers: 2000
 - Products: 200
 - Addresses: 2000

🔹 Output Data Records: 5998

3. STORAGE USAGE
--------------------------------------------------
🔹 Source Data Size: 292.3 KB
🔹 Output Data Size: 1166.9 KB

4. RECOMMENDATIONS & NEXT STEPS
--------------------------------------------------
🔹 Consider adding more data validation checks in pipelines
🔹 Implement automated testing for pipeline components
🔹 Add monitoring for data quality metrics over time
🔹 Consider partitioning output data for large datasets
🔹 Implement error notification system 

In [44]:
# Cell 10: Download Dashboard Results (Optional)
def download_dashboard_results():
    """Package and download dashboard results"""

    print("\n⏬ DOWNLOAD DASHBOARD RESULTS")
    print("="*50)

    # Create report directory
    report_dir = '/content/report'
    os.makedirs(report_dir, exist_ok=True)

    # Save summary statistics
    summary_stats = create_executive_summary()
    with open(f'{report_dir}/summary.json', 'w') as f:
        json.dump(summary_stats, f, indent=2)

    # Save output analysis
    if output_stats is not None:
        output_stats.to_csv(f'{report_dir}/output_analysis.csv', index=False)

    # Package all logs
    shutil.copytree('/content/logs', f'{report_dir}/logs', dirs_exist_ok=True)

    # Create zip file
    shutil.make_archive('/content/dashboard_report', 'zip', report_dir)

    # Download
    files.download('/content/dashboard_report.zip')
    print("✅ Dashboard report downloaded successfully")


In [45]:
download_dashboard_results()

print("\n🎉 PROJECT DASHBOARD COMPLETED SUCCESSFULLY!")
print("="*50)
print("You can now review all pipeline executions and results.")
print("To share your findings, use the download function above.")



⏬ DOWNLOAD DASHBOARD RESULTS
📊 EXECUTIVE SUMMARY
📈 Total Output Files Generated: 5
📈 Pipelines Executed: 3/3
📈 File Formats:
  - .parquet: 1 files
  - .json: 4 files
📈 Total Data Processed: 1166.9 KB


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Dashboard report downloaded successfully

🎉 PROJECT DASHBOARD COMPLETED SUCCESSFULLY!
You can now review all pipeline executions and results.
To share your findings, use the download function above.
